#######################################################################################################

Executes daly to update all the stock signals 
Current signals -
1. EMA Cross indicator
2. Turn Around Stocks

#######################################################################################################


In [1]:
import os, sys

# Get repo root (works both locally & on GitHub Actions)
repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_path = os.path.join(repo_root, "src")

if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ Added to path:", src_path)
print("Current working directory:", os.getcwd())

# Now your imports will work
import get_stocks_data as gs

✅ Added to path: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\src
Current working directory: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\notebooks


In [2]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',120)
import datetime as dt
import datetime
import time 
import ta
import sqlite3

In [3]:
analysis_period = 30 # past 30 days for Analysis
raw_data_needed = analysis_period + 1*365  # +1 year of buffer for calculating indicators
output_period = 30 # restrict output file for EMA-signals in 30 days

In [7]:
# Determine repository root
# GitHub sets GITHUB_WORKSPACE; locally, we use current working directory
repo_root = os.environ.get("GITHUB_WORKSPACE", os.getcwd())
if repo_root == os.getcwd(): #then go one directory above to reach repo root
    repo_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print("Repository root determined as:", repo_root)

# Construct full path to the database
db_path = os.path.join(repo_root, "data", "stocks.db")

# Optional: check if it exists
if not os.path.exists(db_path):
    print(f"ERROR: Database not found at {db_path}")
    sys.exit(1)  # stop execution if DB missing

print("Using database at:", db_path)

# Now you can safely call your function
# Example:
# stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')


Repository root determined as: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals
Using database at: d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db


In [8]:

stocklist = gs.get_company_symbols(db_path, 'COMPANY_DETAILS')
stocklist.head()

tickers = list(stocklist.ticker.unique())
print(len(tickers))
# tickers = ["RELIANCE.NS", "HDFCBANK.NS", "TCS.NS"]

stock_df = gs.fetch_stock_data(tickers, raw_data_needed)

stock_df.head()

2025-10-17 16:34:41,974 - INFO - 📈 Fetching data for 2178 tickers...
2025-10-17 16:34:41,985 - INFO - Fetching data from 2024-09-17 to 2025-10-17


d:\Work\Projects\Stock_Market\Stock_Market\Stock_signals\Stock-Signals\data\stocks.db
2178


[*********************100%***********************]  2178 of 2178 completed
2025-10-17 16:36:22,533 - ERROR - 
1 Failed download:
2025-10-17 16:36:22,534 - ERROR - ['NDTV-RE.NS']: YFInvalidPeriodError("NDTV-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
2025-10-17 16:36:25,655 - INFO - ✅ Download completed in 103.67 seconds
2025-10-17 16:36:31,816 - INFO - 📊 Final dataset: 563,101 rows from 2024-09-17 to 2025-10-17


Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,20ema,50ema,100ema,200ema,30dv,7dv,Symbol
0,2024-09-17,20MICRONS.NS,NaN,294.071899,301.339097,293.375065,301.339097,127357.0,294.071899,294.071899,294.071899,294.071899,NaN,NaN,20MICRONS
1,2024-09-18,20MICRONS.NS,NaN,290.189453,305.818880,288.895310,294.370583,209029.0,293.702143,293.919647,293.995019,294.033268,NaN,NaN,20MICRONS
2,2024-09-19,20MICRONS.NS,NaN,302.683014,305.619763,284.116868,294.669218,246937.0,294.557464,294.263308,294.167059,294.119335,NaN,NaN,20MICRONS
3,2024-09-20,20MICRONS.NS,NaN,300.293793,312.588274,298.651217,304.873117,289918.0,295.103781,294.499798,294.288380,294.180773,NaN,NaN,20MICRONS
4,2024-09-23,20MICRONS.NS,NaN,297.705475,306.067733,296.709971,303.180747,101611.0,295.351561,294.625511,294.356045,294.215844,NaN,NaN,20MICRONS


##### Call Accumulation signal

In [ ]:
import importlib
import accumulation_signal as acs
importlib.reload(acs)

# Detect accumulation
accumulation_df = acs.detect_gradual_accumulation(stock_df)

# Filter the ones meeting entry criteria
accum_df = acs.filter_accumulation(accumulation_df)


status  = acs.save_accumulation_to_db(accum_df, db_path)
if status:
    print("✅ Database updated successfully")
else:
    print("⚠️ Database Update failed")
# View
accum_df.head()

,Symbol,Date,AD_Slope,Above_20EMA_%,Avg_Volume_Spike,RSI,Accumulating,20ema,50ema,200ema,Close,7dv,30dv
0,INFIBEAM,2025-10-14,2.197781e+06,85.0,1.63,68.04,True,17.490480,16.878134,19.053079,18.639999,3.047770e+07,1.384773e+07
1,MRPL,2025-10-14,6.503044e+05,90.0,1.68,65.26,True,137.770038,134.208113,139.141708,141.009995,5.727355e+06,3.516882e+06
2,TICL,2025-10-14,3.582739e+05,50.0,1.78,74.29,False,26.877594,26.551229,31.733777,30.459999,1.380451e+06,7.439027e+05
3,ORCHASP,2025-10-14,2.976880e+05,55.0,1.65,88.30,False,3.024615,2.899069,3.049188,3.890000,1.392519e+06,6.104664e+05
4,SKYGOLD,2025-10-14,2.823914e+05,70.0,1.51,63.46,False,302.231587,298.215153,314.914336,324.549988,2.125439e+06,9.534426e+05


#### EMA Signal indicator - TBC

##### Update the Stock returns table

In [ ]:
# Update the signal returns 
# import accumulation_signal as acs

import importlib
import signal_returns as sigret
# importlib.reload(sigret)

# from signal_returns import generate_signal_returns

status, final_returns = sigret.generate_signal_returns(db_path, stock_df)

if status:
    print("✅ Signal returns generation successful!")
    display(final_returns.head())
else:
    print("❌ Signal returns generation failed.")
